In [1]:
!pip install --quiet wandb
!pip install --quiet nvidia-dali-cuda110
!pip install --quiet lru-dict
!pip install --quiet efficientnet_pytorch
!pip install --quiet pytorch_lightning

## Training Classifier on Heavy Dirt and Clean Images

In [2]:
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

from sys import platform

# Resnet is Resnet 18 (18 Residual SKip Blocks, vs 50)
from models.resnet import ResNet, ResNet50

# These libraries hold dataset information
from dataset import DataloopDataset
# This is the Trainer Library
from training_wheels import TrainingWheels

# Library that allows for to alter "raw" images. Allows for random rotation, flipping, and adding "noise"
from augmentation import Augmentation

from datetime import datetime
from PIL import Image
import matplotlib.pyplot as plt

from argparse import ArgumentParser
import os

In [3]:
os.environ['WANDB_API_KEY']="4a8fc5b45c6c79237bcb32a9659c090b1458dde7"
os.environ['WANDB_DIR'] = '/home/ec2-user/SageMaker/Temp_File_Folder'

In [4]:
hparams = {
    'dataset_dir':"temp",
    'model':"resnet18",
    'use_wandb':True,
    'enable_random_cropping':False,
    'batch_size':32,
    'num_gpus':1,
    'downscaling_width':768,
    'downscaling_height':768,
    'max_epochs':50,
    'accelerator':None,
    'devices':None,
    'use_dali':False,
    'center_crop':448,
    'enable_vertical_mirroring':False,
    'enable_horizontal_mirroring':True,
    'random_rotation_angle':15,
    'noise_amount':0,
    'resume_from_checkpoint':None,
    'enable_image_logging':True,
    'lr':0.00001,
    'balance_sampler':False,
    'train_sample_size':250
}

In [5]:
data_info = {
    'project': "Car Condition Classification",
    'dataset': "all_portrait",
    'email': "ashley.lawrencehuizenga@coxautoinc.com",
    'out_dir': "dataloop/",
    'password': "psych5429!DL"
}

In [6]:
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%m/%d/%Y %H:%M")

In [7]:
dataset_dir = hparams['dataset_dir']

# This loads the augmentation object (see the keywords and how they are used inside )
augmentation = Augmentation(enable_random_cropping=hparams['enable_random_cropping'],
                            enable_vertical_mirroring=hparams['enable_vertical_mirroring'],
                            enable_horizontal_mirroring=hparams['enable_horizontal_mirroring'],
                            random_rotation_angle=hparams['random_rotation_angle'],
                            noise_amount=hparams['noise_amount'],
                            downscaling_width=hparams['downscaling_width'],
                            downscaling_height=hparams['downscaling_height'])

In [8]:
dataset = DataloopDataset(dataset_dir=dataset_dir,
                          email=data_info['email'], 
                          password=data_info['password'], 
                          project=data_info['project'],
                          dataset=data_info['dataset'],
                          train=True,
                          augmentation=augmentation)

Download Items: 100%|██████████| 1/1 [00:00<00:00,  7.23it/s]


In [9]:
batch_size = hparams['batch_size']
logger = WandbLogger(project="car-condition-classifier",log_model='all',name=dt_string) if hparams['use_wandb'] else None
if hparams['model'] == "efficientnet":
    model = EfficientNetV2()
elif hparams['model'] == "resnet18":
    model = ResNet()
elif hparams['model'] == "resnet50":
    model = ResNet50()
else:
    assert False, "Unknown model: {}".format(hparams['model'])
accelerator = "gpu" if hparams['num_gpus'] > 0 else None

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 492MB/s]


In [10]:
if hparams['use_dali']:
    training_wheels = TrainingWheelsDALI(model=model,
                                         dataset=dataset,
                                         batch_size=batch_size,
                                         lr=hparams['lr'])
    if 'PL_TRAINER_GPUS' in os.environ:
        os.environ.pop('PL_TRAINER_GPUS')
    trainer = pl.Trainer(max_epochs=int(hparams['max_epochs']), logger=logger, accelerator=accelerator,
                         devices=max(hparams['num_gpus'], 1))
else:
    training_wheels = TrainingWheels(model=model,
                                     dataset=dataset,
                                     batch_size=batch_size,
                                     augmentation=augmentation,
                                     lr=hparams['lr'],
                                     enable_image_logging=hparams['enable_image_logging'],
                                     validation_set_size=0.2,
                                     balance_sampler=hparams['balance_sampler'],
                                     balance_sample_size=hparams['train_sample_size']
                                    )
    trainer = pl.Trainer(max_epochs=int(hparams['max_epochs']), logger=logger, accelerator=accelerator,
                         devices=max(hparams['num_gpus'], 1))

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(training_wheels, ckpt_path=hparams['resume_from_checkpoint'])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ashley-lawrencehuizenga (fyusion). Use `wandb login --relogin` to force relogin
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
wandb.finish()